# Master the Most Hated Task in DS/ML
## Cleaning categorical data
<img src='images/mop.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@pixabay?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pixabay</a>
        on 
        <a href='https://www.pexels.com/photo/brown-wooden-floor-48889/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

### Introduction

### Setup

In [51]:
# Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Categorical Data, understanding and examples

The formal definition of categorical data would be:
> A predefined set of possible categories or groups an observation can fall into.

You can see examples of categorical data in almost all the datasets you have worked on. Almost any type of data can be turned into categorical. For example:
- Survey responses:
    * `Yes` or `No`
    * `male` or `female`
    * `employed` or `unemployed`
- Numeric data:
    * Annual income in groups: `0-40k`, `40-100k`, ...
    * Ages: child, teenager, adult ...

As we are learning data cleaning using `panads` library, it is important to understand that `pandas` will never import categorical data as categorical. It is mostly imported as strings or integers:

In [52]:
diamonds = pd.read_csv('data/diamonds.csv', index_col=0)
diamonds.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price        int64
x          float64
y          float64
z          float64
dtype: object

You can see that `cut`, `color` and `clarity` are imported as strings rather than as categorical. We could have used the `dtype` parameter of `read_csv` like this:

In [53]:
diamonds = pd.read_csv('data/diamonds.csv', dtype={'cut': 'category', 'color': 'category', 'clarity': 'category'}, index_col=0)
diamonds.dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
price         int64
x           float64
y           float64
z           float64
dtype: object

But with real-world datasets, you often do not have this luxury because the data you will be working probably will have many categorical variables and often, you will be completely unfamiliar with the data. 

Right after you identify the categorical variables, there are some checks and cleaning to be done before you convert the columns to categorical.

### Dealing With Categorical Data Problems

When you work with real-world data, it will be filled with cleaning problems.

As I wrote in the first part of the series, people collecting data won't take into account the cleanliness of the data and do what it takes to record the necessary information in an easy manner as possible. 

Also, problems will arise because of free text during the collection process which leads to typos, many representations of the same value, etc. It is also possible that errors are introduced because of data parsing errors or bad database design.

For example, consider this worst-case scenario: you are working on a survey data conducted across USA and there is a `state` column for the state of each observation in the dataset. There are 50 states in the USA and imagine all the damn variations of state names people can come up with. You are in even bigger problem if data collectors decide to use abbreviations:
- CA, ca, Ca, Caliphornia, Californa, Calfornia, calipornia, CAL, CALI, ...

Such columns will always be filled with typos, errors, inconsistencies. Never imagine that you will have a smooth one-to-one mapping of categories. 

Before moving on to analysis, you have to establish what is called membership constraints which clearly defines the number of categories and how they are represented in a single format.

### Membership Constraints

There are 3 ways you can treat categorical data problems:
- dropping 
- remapping categories
- inferring categories

First, we will focus on isolating inconsistent observations and dropping them. I have created a fake data to illustrate how it is done in code:

In [61]:
# Create possible values for the fake date
first_names = [
    'Lane', 'Ivor', 'Roary', 'Shannon', 
    'Abdul', 'Mary', 'Cole', 'Desirae'
]

last_names = [
  'Reese', 'Pierce', 'Gibson', 'Little',
  'Fry', 'Colon', 'Palmer'
]

blood_types = ['O+', 'O-', 'B+', 'B-', 'A+', 'A-', 'AB+', 'AB-', 'C+', 'D-']
probabilites = [0.12, 0.13, 0.13, 0.12, 0.13, 0.12, 0.12, 0.12, 0.005, 0.005]
size = 10000

dirty_dict = {
    'first_name': np.random.choice(first_names, size),
    'last_name': np.random.choice(last_names, size),
    'blood_type': np.random.choice(blood_types, size, p=probabilites)
}

# Data frame from dirty dict with size 10k
sample = pd.DataFrame(dirty_dict)
sample.to_csv('data/incorrect_blood.csv', index=False)

In [63]:
sample = pd.read_csv('data/incorrect_blood.csv')
sample.head(5)

,first_name,last_name,blood_type
0,Lane,Reese,A+
1,Desirae,Colon,AB+
2,Cole,Little,B+
3,Lane,Palmer,AB-
4,Shannon,Gibson,A+


It is often the case that you will have some background information about your data. For example, let's say you want to check for inconsistencies in the `blood_type` column. You find out beforehand that `blood_type`  can only have these categories: **\[A+, A-, B+, B-, O+, O-, AB+, AB-\]**. So, you have to make sure that the column in the data source only includes these values. 

In our case, there are 10k rows and visual search for inconsistencies is not an option, which is also the case for many other real-world data. Here is how can implement the best solution for such problems:

First, you should create a new data frame which holds all the possible values for a categorical column:

In [55]:
categories = pd.DataFrame({
    'blood_type': ['O+', 'O-', 'B+', 'B-', 'A+', 'A-', 'AB+', 'AB-']
})
categories

,blood_type
0,O+
1,O-
2,B+
3,B-
4,A+
5,A-
6,AB+
7,AB-


> It is a good practice to create such data frames which hold category mappings for each categorical column in the main data.

As we now have the correct categories in a separate data frame, we can use a basic set operation which gives us the difference of unique values in the two columns:

In [56]:
# Create a set of unique blood types of the main data frame
unique_types_main = set(sample['blood_type'].unique())
inconsistent_cats = unique_types_main.difference(categories['blood_type'])
inconsistent_cats

{'C+', 'D-'}

To get the difference of two sets, we use `.difference` function. It basically returns all the values from the left set that are not in the right set. Here is a very simple example:

In [57]:
set_1 = {'A', 'B', 'C', 'D'}
set_2 = {'C', 'D', 'E'}
set_1.difference(set_2)

{'A', 'B'}

Attentive readers may have noticed that inside the `set` function, I also called `.unique()` on the `blood_type`. From what I have read from one StackOverflow thread, it seems that the time it takes to get the unique values will be much shorter if you use both `set` and `unique` for larger datasets. 

Now, we will filter our main data for the blood types 'C+' and 'D-':

In [58]:
inconsistent = sample['blood_type'].isin(inconsistent_cats)
inconsistent

0       False
1       False
2       False
3       False
4       False
        ...  
9995    False
9996    False
9997    False
9998    False
9999    False
Name: blood_type, Length: 10000, dtype: bool

Using `isin` on `blood_type` will return a boolean series which we can use to index the data frame:

In [59]:
inconsistent_rows = sample[inconsistent]
inconsistent_rows

,first_name,last_name,blood_type
74,Ivor,Reese,D-
135,Mary,Gibson,C+
489,Cole,Pierce,D-
573,Lane,Pierce,C+
574,Ivor,Little,D-
...,...,...,...
8828,Ivor,Gibson,D-
9046,Abdul,Little,D-
9283,Shannon,Fry,C+
9293,Cole,Reese,D-


So, there are 142 individuals with incorrect blood types. Since we don't have any insight of how these errors occurred, we have to drop them. It can be done in two ways:

In [60]:
# Using drop
consistent = sample.drop(inconsistent_rows.index)
# Using negative conditioning
consistent = sample[~inconsistent]
# Check the results
assert set(consistent['blood_type'].unique()) == set(categories['blood_type'])